## P3P torch implementation in batches

In [89]:
import numpy as np 
import torch

In [90]:
# Define the dtype and device for PyTorch tensors
# This allows for easy switching between CPU and GPU computations
dtype = torch.float32
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 16

At first we have : 
- first camera parameters -> A (focal, etc) 
- rotation  -> R 
- position - C ou T ?  matrix
- 3D points position ->  P1 P2 P3 

In [91]:
def camera(batch_size, device):
    fx = 800.0
    fy = 800.0
    cx = 320.0
    cy = 240.0

    A_single = torch.tensor([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1]
    ], dtype=torch.float32, device=device)

    A = A_single.unsqueeze(0).repeat(batch_size, 1, 1)
    return A

def rotation_matrix(batch_size, device):
    R_single = torch.tensor([
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1]
    ], dtype=torch.float32, device=device)

    R = R_single.unsqueeze(0).repeat(batch_size, 1, 1)
    return R

def camera_position(batch_size, device):
    C_single = torch.tensor([[0, 0, 6]], dtype=torch.float32, device=device)
    C = C_single.repeat(batch_size, 1, 1)  # (B, 1, 3)
    return C


A = camera(batch_size, device)
R = rotation_matrix(batch_size, device)
C = camera_position(batch_size, device)

print("A shape:", A.shape)   # (B, 3, 3)
print("R shape:", R.shape)   # (B, 3, 3)
print("C shape:", C.shape)   # (B, 1, 3)
print("A dtype:", A.dtype)   # torch.float32

A shape: torch.Size([16, 3, 3])
R shape: torch.Size([16, 3, 3])
C shape: torch.Size([16, 1, 3])
A dtype: torch.float32


here in batch we want : 
- (B,3,3) for intrisic camera param A
- (B,3,3) for rotation matrix R
- (B,1,3) for camera position C

In [92]:
# Definition of 3D points in the world coordinate system
def point3Daleatoire(x, batch_size, device):
    # Generation of random points in 3D space
    # Output shape: (B, 3)
    points = torch.tensor(
        np.random.uniform(-x, x, size=(batch_size, 3)),
        dtype=torch.float32,
        device=device
    )
    return points

def pts_3D_4pts(batch_size, device):
    # Generate randomly 4 3D points for each batch
    # Output: array which concatenates the 4 points = [P1, P2, P3, P4]
    # Shape = (B, 4, 3)

    P1 = point3Daleatoire(2, batch_size, device)    # (B, 3)
    P2 = point3Daleatoire(2, batch_size, device)
    P3 = point3Daleatoire(2, batch_size, device)
    P4 = point3Daleatoire(2, batch_size, device)

    # Stack into (B, 4, 3)
    points3D = torch.stack((P1, P2, P3, P4), dim=1)

    print("points3D =\n", points3D)
    print("points3D.shape =", points3D.shape)  # (B, 4, 3)
    return points3D 

In [93]:
P1 = torch.tensor([0.7161, 0.5431, 1.7807], dtype=torch.float64)
P2 = torch.tensor([-1.1643, 0.8371, -1.0551], dtype=torch.float64)
P3 = torch.tensor([-1.5224, 0.4292, -0.1994], dtype=torch.float64)
P4 = torch.tensor([-1.5224, 0.4292, -0.1994], dtype=torch.float64)

In [94]:
# Single set of fixed points as a tensor (4, 3)
fixed_points_single = torch.stack([P1, P2, P3, P4], dim=0)

# Convert to float32 and move to device
fixed_points_single = fixed_points_single.to(dtype=torch.float32, device=device)

# Repeat across batch → shape (B, 4, 3)
points3D_batch = fixed_points_single.unsqueeze(0).repeat(batch_size, 1, 1)

print("fixed_points_batch.shape =", points3D_batch.shape)  # (B, 4, 3)
print("fixed_points_batch =\n", points3D_batch)

fixed_points_batch.shape = torch.Size([16, 4, 3])
fixed_points_batch =
 tensor([[[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

    

We create the 3 direction features vectors f1, f2, f3

In [95]:
def features_vectors(points3D, C, R, batch_size, device):
    '''
    This function computes the features vectors for P3P algorithm.
    args:
    points3D : array with the 4 3D points = [ P1, P2, P3, P4 ] (B, 4, 3)
    C: camera position matrix : (B, 1, 3)
    R: rotation matrix : (B, 3, 3)
    returns:
    featuresVect : array with the features vectors (B, 9)
    '''
    
    # Extract the first three points from points3D
    P1 = points3D[:, 0, :]  # (B, 3)
    P2 = points3D[:, 1, :]  # (B, 3)
    P3 = points3D[:, 2, :]  # (B, 3)

    C = C.squeeze(1)  # Remove the second dimension to get (B, 3)

    # Compute vectors from camera center to each point
    v1 = torch.bmm(R, (P1 - C).unsqueeze(-1)).squeeze(-1)  # (B, 3)
    v2 = torch.bmm(R, (P2 - C).unsqueeze(-1)).squeeze(-1)  # (B, 3)
    v3 = torch.bmm(R, (P3 - C).unsqueeze(-1)).squeeze(-1)  # (B, 3)


    f1 = v1 / torch.norm(v1, dim=1, keepdim=True)  # Normalize v1
    f2 = v2 / torch.norm(v2, dim=1, keepdim=True)  # Normalize v2
    f3 = v3 / torch.norm(v3, dim=1, keepdim=True)  # Normalize v3

    # Stack into matrix (B, 3, 3)
    featuresVect = torch.stack([f1, f2, f3], dim=1)    # shape (B, 3, 3)


    print("features vectors shape =", featuresVect.shape)  # Should be (B, 9)
    
    return featuresVect
    

featuresVect = features_vectors(points3D_batch, C, R, batch_size, device)
print("featuresVect =\n", featuresVect)
print(featuresVect.shape) 

features vectors shape = torch.Size([16, 3, 3])
featuresVect =
 tensor([[[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        

Lastly we need the functions to resolve the polynomial roots. - for test go to test resolution polynome 

In [96]:
from complex_utils_batch import *

def polynomial_root_calculation_4th_degree_ferrari_batch(a_batch):
    """
    Solves a batch of quartic polynomials using Ferrari's method.

    Args:
        a_batch → shape (B, 5)
          rows: [a0, a1, a2, a3, a4]
    
    Returns:
        roots_batch → shape (B, 4, 2)
            each root stored as [real, imag]
    """

    B = a_batch.shape[0]

    # Unpack coefficients
    a0 = a_batch[:, 0]
    a1 = a_batch[:, 1]
    a2 = a_batch[:, 2]
    a3 = a_batch[:, 3]
    a4 = a_batch[:, 4]

    # Reduce to x⁴ + a·x³ + b·x² + c·x + d = 0
    a = a3 / a4
    b = a2 / a4
    c = a1 / a4
    d = a0 / a4

    S = a / 4
    b0 = d - c*S + b*S**2 - 3*S**4
    b1 = c - 2*b*S + 8*S**3
    b2 = b - 6*S**2

    # Solve cubic in batch
    x_cube = polynomial_root_calculation_3rd_degree_batch(
        torch.ones_like(b2),
        b2,
        (b2**2)/4 - b0,
        -b1**2/8
    )                           # shape (B, 3, 2)

    # Find real and positive solutions in batch
    real_mask = torch.isclose(x_cube[:, :, 1], torch.zeros_like(x_cube[:, :, 1]), atol=1e-7)
    pos_mask = x_cube[:, :, 0] > 0
    select_mask = real_mask & pos_mask

    alpha0_real = torch.zeros(B, device=a_batch.device)
    alpha0_imag = torch.zeros(B, device=a_batch.device)

    for i in range(B):
        idxs = torch.where(select_mask[i])[0]
        if len(idxs) > 0:
            idx = idxs[0]
            alpha0_real[i] = x_cube[i, idx, 0]
            alpha0_imag[i] = 0.0
        else:
            alpha0_real[i] = 0.0
            alpha0_imag[i] = float('nan')

    alpha0 = torch.stack([alpha0_real, alpha0_imag], dim=1)    # shape (B, 2)

    is_real_branch = torch.isclose(alpha0[:, 1], torch.zeros_like(alpha0[:, 1]), atol=1e-7)

    # Initialize roots array
    roots_batch = torch.zeros((B, 4, 2), device=a_batch.device)

    # Branch 1: alpha0 real and positive
    if is_real_branch.any():

        mask = is_real_branch
        alpha0_sel = alpha0[mask]
        b2_sel = b2[mask]
        S_sel = S[mask]
        b1_sel = b1[mask]

        alpha0_div_2 = product_complex_real_batch(alpha0_sel, 0.5)
        sqrt_alpha = torch.sqrt(alpha0_div_2[:, 0])      # shape (B,)
        sqrt_alpha_complex = torch.stack([sqrt_alpha, torch.zeros_like(sqrt_alpha)], dim=1)
        print("sqrt_alpha shape:", sqrt_alpha.shape)  # Should be (B,)
        
        term = addition_complex_real_batch(
            -alpha0_div_2,
            -b2_sel / 2
        )

        denom_real = 2 * torch.sqrt(2 * alpha0_sel[:, 0])
        denom = torch.stack([denom_real, torch.zeros_like(denom_real)], dim=1)

        frac = division_2_complex_numbers_batch(
            torch.stack([b1_sel, torch.zeros_like(b1_sel)], dim=1),
            denom
        )

        sqrt_alpha_complex = torch.stack([sqrt_alpha, torch.zeros_like(sqrt_alpha)], dim=1)

        x1 = addition_complex_real_batch(
            sqrt_alpha_complex,
            -S_sel
        )
        x1 = addition_batch(x1, sqrt_complex_batch(addition_complex_real_batch(term, -frac)))

        x2 = addition_complex_real_batch(
            sqrt_alpha.unsqueeze(1),
            -S_sel
        )
        x2 = addition_batch(x2, -sqrt_complex_batch(addition_complex_real_batch(term, -frac)))

        x3 = addition_complex_real_batch(
            -sqrt_alpha.unsqueeze(1),
            -S_sel
        )
        x3 = addition_batch(x3, sqrt_complex_batch(addition_complex_real_batch(term, frac)))

        x4 = addition_complex_real_batch(
            -sqrt_alpha.unsqueeze(1),
            -S_sel
        )
        x4 = addition_batch(x4, -sqrt_complex_batch(addition_complex_real_batch(term, frac)))

        roots_batch[mask, 0, :] = x1
        roots_batch[mask, 1, :] = x2
        roots_batch[mask, 2, :] = x3
        roots_batch[mask, 3, :] = x4

    # Branch 2: no real positive alpha0
    if (~is_real_branch).any():

        mask = ~is_real_branch

        sqrt_inner1 = sqrt_batch((b2[mask]**2)/4 - b0[mask])

        minus_b2_over_2 = -b2[mask] / 2

        x1 = addition_complex_real_batch(
            sqrt_complex_batch(addition_complex_real_batch(sqrt_inner1, minus_b2_over_2)),
            -S[mask]
        )

        x2 = addition_complex_real_batch(
            -sqrt_complex_batch(addition_complex_real_batch(sqrt_inner1, minus_b2_over_2)),
            -S[mask]
        )

        x3 = addition_complex_real_batch(
            sqrt_complex_batch(addition_complex_real_batch(-sqrt_inner1, minus_b2_over_2)),
            -S[mask]
        )

        x4 = addition_complex_real_batch(
            -sqrt_complex_batch(addition_complex_real_batch(-sqrt_inner1, minus_b2_over_2)),
            -S[mask]
        )

        roots_batch[mask, 0, :] = x1
        roots_batch[mask, 1, :] = x2
        roots_batch[mask, 2, :] = x3
        roots_batch[mask, 3, :] = x4

    return roots_batch

We have all the variables needed for the p3p so we start

1. we stock 3D points in batch

In [97]:
print(points3D_batch)
print("points3D_batch.shape =", points3D_batch.shape)  # (B, 4, 3)

tensor([[[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],
         [-1.5224,  0.4292, -0.1994],
         [-1.5224,  0.4292, -0.1994]],

        [[ 0.7161,  0.5431,  1.7807],
         [-1.1643,  0.8371, -1.0551],


2. we stock features vectors in batch also

In [98]:
print("featuresVect =\n", featuresVect)
print("featuresVect.shape =", featuresVect.shape)  # (B, 9)

featuresVect =
 tensor([[[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617, -0.1163,  0.9800],
         [-0.2379, -0.0671,  0.9690]],

        [[ 0.1660, -0.1259,  0.9781],
         [-0.1617,

3. we create the batched solution tensor, we've got at best 4 solution and batch_size groups, we got (16,1,3) C and (16,3,3) R 

In [99]:
solutions = torch.zeros((batch_size,4,3,4), dtype=dtype, device=device)
print("solutions = \n", solutions)
print(solutions.shape)  # (16,4,3,4)

solutions = 
 tensor([[[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]]],


        [[[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
          [0., 0., 0., 0.],
          [0., 0., 0., 0.]],

         [[0., 0., 0., 0.],
        

4. On teste si les 3 points ne sont pas colinéaires

In [100]:
# Test of non-collinearity
def check_collinearity_batch(points3D_batch, eps=1e-8, verbose=True):
    """
    Checks non-collinearity of the first 3 points in a batched set of 3D points.

    Args:
        points3D_batch (torch.Tensor): shape (B, 4, 3)
            Batch of sets of 4 3D points. Only the first three points are checked.
        eps (float): threshold to consider vectors collinear.
        verbose (bool): if True, prints messages for each batch element.

    Returns:
        is_collinear (torch.BoolTensor): shape (B,)
            Boolean tensor indicating whether points in each batch are collinear (True) or not (False).
    """

    # Extract points
    P1 = points3D_batch[:, 0, :]   # (B, 3)
    P2 = points3D_batch[:, 1, :]
    P3 = points3D_batch[:, 2, :]

    # Compute vectors
    v1 = P2 - P1                   # (B, 3)
    v2 = P3 - P1

    # Cross product
    cross_prod = torch.cross(v1, v2, dim=1)     # (B, 3)

    # Norm of cross product
    norm_cross = torch.norm(cross_prod, dim=1)  # (B,)

    # Collinearity test
    is_collinear = norm_cross < eps             # (B,)

    if verbose:
        for i, collinear in enumerate(is_collinear):
            if collinear:
                print(f"Batch {i}: Problem — points are collinear!")
            else:
                print(f"Batch {i}: Points are not collinear, we can continue.")

    return is_collinear

is_collinear = check_collinearity_batch(points3D_batch)

print("is_collinear =", is_collinear)

Batch 0: Points are not collinear, we can continue.
Batch 1: Points are not collinear, we can continue.
Batch 2: Points are not collinear, we can continue.
Batch 3: Points are not collinear, we can continue.
Batch 4: Points are not collinear, we can continue.
Batch 5: Points are not collinear, we can continue.
Batch 6: Points are not collinear, we can continue.
Batch 7: Points are not collinear, we can continue.
Batch 8: Points are not collinear, we can continue.
Batch 9: Points are not collinear, we can continue.
Batch 10: Points are not collinear, we can continue.
Batch 11: Points are not collinear, we can continue.
Batch 12: Points are not collinear, we can continue.
Batch 13: Points are not collinear, we can continue.
Batch 14: Points are not collinear, we can continue.
Batch 15: Points are not collinear, we can continue.
is_collinear = tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False])


5. On crée un repère orthonormée à partir de f1, f2, f3

In [101]:
# Extract feature vectors f1 and f2 from featuresVect
f1 = featuresVect[:,0,:]    # shape (B, 3)
f2 = featuresVect[:,1,:]    # shape (B, 3)

# Compute the base vectors of τ
tx = f1

cross_f1_f2 = torch.cross(f1, f2, dim=1)   # (B, 3)
norm_cross = torch.norm(cross_f1_f2, dim=1, keepdim=True)   # (B, 1)

tz = cross_f1_f2 / (norm_cross)    # (B, 3)

ty = torch.cross(tz, tx, dim=1)          # (B, 3)

# assemble τ as a (B, 3, 3) matrix
T = torch.stack([tx, ty, tz], dim=1)   # (B, 3, 3)

# Print results
print("tx =\n", tx)
print("tx.shape =", tx.shape)    # (B, 3)

print("tz =\n", tz)
print("tz.shape =", tz.shape)    # (B, 3)

print("ty =\n", ty)
print("ty.shape =", ty.shape)    # (B, 3)

print("tau =\n", T)
print("tau.shape =", T.shape)  # (B, 3, 3)

tx =
 tensor([[ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781],
        [ 0.1660, -0.1259,  0.9781]])
tx.shape = torch.Size([16, 3])
tz =
 tensor([[-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1226],
        [-0.0298, -0.9920, -0.1

5.bis On crée la matrice de transformation T  

In [102]:
f3 = featuresVect[:, 2, :].unsqueeze(2)   # (B, 3, 1)

f3_T = torch.bmm(T,f3) 
print("f3_T = \n", f3_T)
print(f3_T.shape)  # (B,3,1)
print("T = \n", T)
print(T.shape)  # (B,3,3)

f3_T = 
 tensor([[[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]],

        [[ 0.9166],
         [ 0.3971],
         [-0.0452]]])

Il faut satisfaire la condition que z dans f3_T est négative, sinon on inverse l'ordre des vecteur afin de .... 

In [103]:
f3_T_positif = False

# Having teta in [ 0, pi ] 
# cause f3_T[:,2] is the z component of the vector f3_T
# like that tensor([ True, False, True, ... ])
if (f3_T[:,2] > 0).any() :
    f3_T_positif = True
    print("f3_T_positif =", f3_T_positif)   
   

6. on change de base du coté des points 3D

In [104]:
P1 = points3D_batch[:, 0, :]  # (B, 3)
P2 = points3D_batch[:, 1, :]  # (B, 3)
P3 = points3D_batch[:, 2, :]  # (B, 3


# Compute nx
v1 = P2 - P1                            # (B, 3)
norm_v1 = torch.norm(v1, dim=1, keepdim=True) + 1e-8
nx = v1 / norm_v1                       # (B, 3)

# Compute nz
v2 = P3 - P1                            # (B, 3)
cross_v1_v2 = torch.cross(nx, v2, dim=1)
norm_cross = torch.norm(cross_v1_v2, dim=1, keepdim=True) + 1e-8
nz = cross_v1_v2 / norm_cross           # (B, 3)

# Compute ny
ny = torch.cross(nz, nx, dim=1)         # (B, 3)

print("nx =\n", nx)
print("nx.shape =", nx.shape)           # (B, 3)
print("ny =\n", ny)
print("ny.shape =", ny.shape)
print("nz =\n", nz)
print("nz.shape =", nz.shape)

# Build N matrix
# First reshape each vector to (B, 1, 3)
nx_row = nx.unsqueeze(1)                # (B, 1, 3)
ny_row = ny.unsqueeze(1)
nz_row = nz.unsqueeze(1)

# Concatenate rows → (B, 3, 3)
N = torch.cat((nx_row, ny_row, nz_row), dim=1)

print("N =\n", N)
print("N.shape =", N.shape)             # (B, 3, 3)

# Compute P3 - P1
P3_minus_P1 = (P3 - P1).unsqueeze(2)    # (B, 3, 1)

# Compute P3_n = N * (P3 - P1)
P3_n = torch.bmm(N, P3_minus_P1)        # (B, 3, 1)
P3_n = P3_n.squeeze(2)                  # (B, 3)

print("P3_n =\n", P3_n)
print("P3_n.shape =", P3_n.shape)       # (B, 3)

nx =
 tensor([[-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303],
        [-0.5506,  0.0861, -0.8303]])
nx.shape = torch.Size([16, 3])
ny =
 tensor([[-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4698],
        [-0.7747, -0.4233,  0.4

7. On définit des variable pour la suite

In [105]:
# Computation of phi1 et phi2 with 0=x, 1=y, 2=z

phi1 = f3_T[:,0]/f3_T[:,2]
phi2 = f3_T[:,1]/f3_T[:,2]
print("phi1 = ", phi1)
print("phi2 = ", phi2)
print("phi1.shape =", phi1.shape)  # (B,)
print("phi2.shape =", phi2.shape)  # (B,)

# Extraction of p1 and p2 from P3_eta
p1 = P3_n[:,0] #x
p2 = P3_n[:,1] #y
p1 = p1.unsqueeze(1)
p2 = p2.unsqueeze(1)
print("p1 = ", p1)
print("p2 = ", p2)
print("p1.shape =", p1.shape)  # (B,)
print("p2.shape =", p2.shape)  # (B,)   

# Computation of d12
d12 = torch.norm(P2-P1, dim=1, keepdim=True)  # (B, 1)
print("d12 = ", d12)
print("d12.shape =", d12.shape)  # (B, 1)


phi1 =  tensor([[-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913],
        [-20.2913]])
phi2 =  tensor([[-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914],
        [-8.7914]])
phi1.shape = torch.Size([16, 1])
phi2.shape = torch.Size([16, 1])
p1 =  tensor([[2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668],
        [2.8668]

In [106]:
# f1, f2 → shape (B, 3)

dot_f1_f2 = torch.sum(f1 * f2, dim=1)             # (B,)
norm_f1 = torch.norm(f1, dim=1)                   # (B,)
norm_f2 = torch.norm(f2, dim=1)                   # (B,)

cosBeta = dot_f1_f2 / (norm_f1 * norm_f2 + 1e-8)  # (B,)

# Compute cot(beta)
b = torch.sqrt(1. / (1. - cosBeta**2 + 1e-8) - 1.)   # (B,)

# Flip sign if cosBeta < 0
b = torch.where(cosBeta < 0, -b, b)

# Optionally reshape to (B, 1)
cosBeta = cosBeta.unsqueeze(1)
b = b.unsqueeze(1)

print("cosBeta =", cosBeta)
print("b =", b)

cosBeta = tensor([[0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463],
        [0.9463]])
b = tensor([[2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257],
        [2.9257]])


In [107]:
a4 = - phi2**2 * p2**4 - phi1**2 * p2**4 - p2**4
a3 = 2 * p2**3 * d12 * b + 2 * phi2**2 * p2**3 * d12 * b - 2 * phi1 * phi2 * p2**3 * d12
a2 = - phi2**2 * p1**2 * p2**2 - phi2**2 * p2**2 * d12**2 * b**2 - phi2**2 * p2**2 * d12**2 + phi2**2 * p2**4 + phi1**2 * p2 **4 + 2 * p1 * p2**2 * d12 + 2 * phi1 * phi2 * p1 * p2**2 * d12 * b - phi1**2 * p1**2 * p2**2 + 2 * phi2**2 * p1 * p2**2 * d12 - p2**2 * d12**2 * b**2 - 2 * p1**2 * p2**2
a1 = 2 * p1**2 * p2 * d12 * b + 2 * phi1 * phi2 * p2**3 * d12 - 2 * phi2**2 * p2**3 * d12 * b - 2 * p1 * p2 * d12**2 * b
a0 = - 2 * phi1 * phi2 * p1 * p2**2 * d12 * b + phi2**2 * p2**2 * d12**2 + 2 * p1**3 * d12 - p1**2 * d12**2 + phi2**2 * p1**2 * p2**2 - p1**4 - 2 * phi2**2 * p1 * p2**2 * d12 + phi1**2 * p1**2 * p2**2 + phi2**2 * p2**2 * d12**2 * b**2

print("a4 = ", a4)
print("a3 = ", a3)
print("a2 = ", a2)
print("a1 = ", a1) 
print("a0 = ", a0)
print("a4.shape =", a4.shape)  # (B,)


a4 =  tensor([[-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022],
        [-258.3022]])
a3 =  tensor([[214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644],
        [214.0644]])
a2 =  tensor([[-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [-468.5417],
        [

In [108]:
# Stack all your coefficients into a_batch → shape (16, 5)
a_batch = torch.cat([a0, a1, a2, a3, a4], dim=1)     # shape (16,5)
roots_batch = polynomial_root_calculation_4th_degree_ferrari_batch(a_batch)

sqrt_alpha shape: torch.Size([16])


RuntimeError: The size of tensor a (16) must match the size of tensor b (2) at non-singleton dimension 1

In [69]:
# For each solution of the polynomial
for i in range(4):
  #if np.isclose(np.imag(roots[i]),0) : # if real solution 

    # Computation of trigonometrics forms
    cos_teta = torch.tensor(roots[i][0])# real part of the root 
    sin_teta = torch.sqrt(1-cos_teta**2)

    cot_alpha = ((phi1/phi2)*p1 + cos_teta*p2 -d12*b )/ ((phi1/phi2)*cos_teta* p2 - p1 + d12)

    sin_alpha = torch.sqrt(1/(cot_alpha**2+1))
    cos_alpha= torch.sqrt(1-sin_alpha**2)

    if cot_alpha < 0 :
      cos_alpha = -cos_alpha

    # Computation of the intermediate rotation's matrixs
    C_estimate = torch.tensor([d12*cos_alpha*(sin_alpha*b + cos_alpha), d12*sin_alpha*cos_teta*(sin_alpha*b+cos_alpha), d12*sin_alpha*sin_teta*(sin_alpha*b+cos_alpha)]) # (3,)
    print("C_estimate = \n", C_estimate)
    print(C_estimate.shape)  # (3,)
    Q = torch.tensor([[-cos_alpha, -sin_alpha*cos_teta, -sin_alpha*sin_teta], [sin_alpha, -cos_alpha*cos_teta, -cos_alpha*sin_teta], [0, -sin_teta, cos_teta]])    # (3*3)
    print("Q = \n", Q)
    print(Q.shape)  # (3,3)
    # Computation of the absolute camera center
  
    C_estimate = P1 + torch.tensordot(torch.transpose(N, 0,1), C_estimate, dims=1) # (3,)
    print("C_estimate = \n", C_estimate) 
    print(C_estimate.shape)  # (3,)
    C_estimate = torch.reshape(C_estimate, (3,1))  # Reshape to (3,1) for consistency
    print("C_estimate = \n", C_estimate)  # (3,1)
    print("C_estimate.shape = ", C_estimate.shape)  # (3,1)

    # Computation of the orientation matrix
    R_estimate = torch.tensordot(torch.tensordot(torch.transpose(N,0,1),torch.transpose(Q, 0,1), dims=1),T,dims=1)   # (3*3)
    print("R_estimate = \n", R_estimate)
    print(R_estimate.shape)  # (3,3)
    
    # Adding C and R to the solutions
    solutions[i,:,:1]= C_estimate
    solutions[i,:,1:] = R_estimate

print("solutions = \n", solutions)


C_estimate = 
 tensor([-1.0076,  0.2018,  0.4222], dtype=torch.float64)
torch.Size([3])
Q = 
 tensor([[ 0.9070, -0.1817, -0.3800],
        [ 0.4212,  0.3912,  0.8183],
        [ 0.0000, -0.9022,  0.4313]], dtype=torch.float64)
torch.Size([3, 3])
C_estimate = 
 tensor([0.9832, 0.7517, 2.8388], dtype=torch.float64)
torch.Size([3])
C_estimate = 
 tensor([[0.9832],
        [0.7517],
        [2.8388]], dtype=torch.float64)
C_estimate.shape =  torch.Size([3, 1])
R_estimate = 
 tensor([[ 0.7214, -0.5369, -0.4374],
        [-0.6541, -0.7357, -0.1756],
        [-0.2275,  0.4128, -0.8820]], dtype=torch.float64)
torch.Size([3, 3])
C_estimate = 
 tensor([-1.0076,  0.2018,  0.4222], dtype=torch.float64)
torch.Size([3])
Q = 
 tensor([[ 0.9070, -0.1817, -0.3800],
        [ 0.4212,  0.3912,  0.8183],
        [ 0.0000, -0.9022,  0.4313]], dtype=torch.float64)
torch.Size([3, 3])
C_estimate = 
 tensor([0.9832, 0.7517, 2.8388], dtype=torch.float64)
torch.Size([3])
C_estimate = 
 tensor([[0.9832],
        

C:\Users\pmamalet\AppData\Local\Temp\ipykernel_18092\4123210296.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cos_teta = torch.tensor(roots[i][0])# real part of the root


In [74]:
print("solutions = \n", solutions)
print(solutions.shape)  # (4,3,4)

solutions = 
 tensor([[[ 9.8322e-01,  7.2140e-01, -5.3692e-01, -4.3738e-01],
         [ 7.5168e-01, -6.5409e-01, -7.3574e-01, -1.7564e-01],
         [ 2.8388e+00, -2.2749e-01,  4.1279e-01, -8.8196e-01]],

        [[ 9.8322e-01,  7.2140e-01, -5.3692e-01, -4.3738e-01],
         [ 7.5168e-01, -6.5409e-01, -7.3574e-01, -1.7564e-01],
         [ 2.8388e+00, -2.2749e-01,  4.1279e-01, -8.8196e-01]],

        [[ 6.8834e-15,  1.0000e+00, -3.7068e-15, -1.7408e-15],
         [ 1.4877e-14, -3.6966e-15, -1.0000e+00, -2.5909e-15],
         [ 6.0000e+00, -1.7123e-15,  2.6610e-15, -1.0000e+00]],

        [[-1.2586e+00,  8.4783e-01, -5.1920e-01,  1.0778e-01],
         [ 2.2671e+00,  5.2791e-01,  8.0731e-01, -2.6374e-01],
         [-3.9910e+00,  4.9920e-02,  2.8051e-01,  9.5855e-01]]],
       dtype=torch.float64)
torch.Size([4, 3, 4])


In [76]:
def projection3D2D(point3D,C,R,A) :
  # 3D point = [ Xw, Yw, Zw ]'   (1*3)
  # T : camera translation matrix : (3*1)
  # R : camera rotation matrix : (3*3)
  # A : intraseca matrix of the camera : (3*3)
  # Output : return the coordonates of the point in 2D 

  PI = torch.cat((torch.eye(3, dtype=torch.float64),torch.zeros((3,1), dtype=torch.float64)),dim=1)  # (3*4)

  Rt = torch.cat((R,C),dim=1)               # (3*4)
  Rt = torch.cat((Rt,torch.tensor([[0,0,0,1]], dtype=torch.float64)),dim=0)   # (4*4)

  point3D_bis = torch.cat((torch.reshape(point3D,(3,1)),torch.tensor([[1]],dtype=torch.float64)),dim=0)   #(4*1)
  
  point2D = torch.tensordot(torch.tensordot(torch.tensordot(A,PI,dims=1),Rt,dims=1),point3D_bis,dims=1)  # 2D point = [u, v, w] (3*1)
  point2D = point2D / point2D[2]        # 2D point = [u, v, 1] (3*1)
  return point2D[:2]


C_transpose = torch.transpose(C, 0, 1)  # (3*1) -> (1*3)

p1 = projection3D2D(P1,C_transpose,R,A)
print("p1 = ", p1)
print(p1.shape)  # (2,1)
p2 = projection3D2D(points3D[1],C_transpose,R,A)
print("p2 = ", p2)
p3 = projection3D2D(points3D[2],C_transpose,R,A)
print("p3 = ", p3)
p4 = projection3D2D(P4,C_transpose,R,A)
print("p4 = ", p4)




p1 =  tensor([[455.7761],
        [137.0256]], dtype=torch.float64)
torch.Size([2, 1])
p2 =  tensor([[187.9764],
        [145.0786]], dtype=torch.float64)
p3 =  tensor([[123.5423],
        [184.6140]], dtype=torch.float64)
p4 =  tensor([[123.5423],
        [184.6140]], dtype=torch.float64)


In [72]:
def distance(pt, pt_estimation):
    erreur = torch.tensor(0, dtype=torch.float64)  # Initialize error as a tensor
    for i in range(len(pt)): 
      erreur = erreur + torch.tensor((pt[i] - pt_estimation[i])**2, dtype=torch.float64)  # Ensure each term is a tensor
      #erreur += (pt[i] - pt_estimation[i])**2
    return torch.sqrt(erreur)



def affichage_erreur(solutions,points2D,points3D,A) : 
   # Compute the error of estimation for each points after the P3P algorithm 

   # solutions : solution matrix returned by P3P (4*3*4)
   # points 3D : 4 pts 3D used for P3P 
   # points 2D : 4 pts 2D used for P3P (image of the 3D points)
   
   P1 = points3D[0]
   P2 = points3D[1]
   P3 = points3D[2]
   P4 = points3D[3]

   erreurs = []
   nb_sol = 0

   for i in range(len(solutions)) : 
      R = solutions[i,:,1:] 
      C = solutions[i,:,:1]

      if not torch.all(R==torch.zeros((3,3))) : 
        nb_sol += 1 
        print("------------ Solution n° : ",nb_sol,"----------------")
        print("R = \n",R,)
        print("T = \n",C,)

        p1_P3P = torch.reshape(projection3D2D(P1,C,R,A),(1,2))
        p2_P3P = torch.reshape(projection3D2D(P2,C,R,A),(1,2))
        p3_P3P = torch.reshape(projection3D2D(P3,C,R,A),(1,2))
        p4_P3P = torch.reshape(projection3D2D(P4,C,R,A),(1,2))
        pt_2D_P3P = torch.cat((p1_P3P,p2_P3P,p3_P3P,p4_P3P),dim=0)    # (4,2)

        erreurs.append([0])
        for j in range(len(points2D)):
            erreur_pt = distance(points2D[j],pt_2D_P3P[j])
            erreurs[i]+=erreur_pt
        
   indice_min = 0
   min = erreurs[0]
   for i in range(1,len(erreurs)) :
    if erreurs[i]<min :
      min = erreurs[i]
      indice_min = i

   R_opti = solutions[indice_min,:,1:] 
   C_opti = solutions[indice_min,:,:1]
   print("\n------------ Best solution : ----------------")
   print("Solution n° :",indice_min+1,"\n")
   print("R estimé = \n", R_opti,"\n")
   print("T estimé = \n", C_opti, "\n")

In [73]:
affichage_erreur(solutions, [p1, p2, p3, p4], [P1,P2,P3, P4], A)

------------ Solution n° :  1 ----------------
R = 
 tensor([[ 0.7214, -0.5369, -0.4374],
        [-0.6541, -0.7357, -0.1756],
        [-0.2275,  0.4128, -0.8820]], dtype=torch.float64)
T = 
 tensor([[0.9832],
        [0.7517],
        [2.8388]], dtype=torch.float64)
------------ Solution n° :  2 ----------------
R = 
 tensor([[ 0.7214, -0.5369, -0.4374],
        [-0.6541, -0.7357, -0.1756],
        [-0.2275,  0.4128, -0.8820]], dtype=torch.float64)
T = 
 tensor([[0.9832],
        [0.7517],
        [2.8388]], dtype=torch.float64)
------------ Solution n° :  3 ----------------
R = 
 tensor([[ 1.0000e+00, -3.7068e-15, -1.7408e-15],
        [-3.6966e-15, -1.0000e+00, -2.5909e-15],
        [-1.7123e-15,  2.6610e-15, -1.0000e+00]], dtype=torch.float64)
T = 
 tensor([[6.8834e-15],
        [1.4877e-14],
        [6.0000e+00]], dtype=torch.float64)
------------ Solution n° :  4 ----------------
R = 
 tensor([[ 0.8478, -0.5192,  0.1078],
        [ 0.5279,  0.8073, -0.2637],
        [ 0.0499,  0.

C:\Users\pmamalet\AppData\Local\Temp\ipykernel_18092\875224362.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  erreur = erreur + torch.tensor((pt[i] - pt_estimation[i])**2, dtype=torch.float64)  # Ensure each term is a tensor
